In [60]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

import sys

sys.path.insert(0,'/Users/orentapiero/btc_research/')

from btc_functions.glassnode import *
from btc_functions.import_data import get_glassnode_price,get_glassnode_data
from btc_functions.variable_list_urls import *
from btc_functions.utilities import strided_app,strided_app2

plt.rcParams['figure.figsize'] = [15, 10]
sns.set()

In [61]:
ohlc = get_glassnode_price()
ohlc = ohlc.rename(columns = {'o':'Open','h':'High','l':'Low','c':'Close'})
features = get_glassnode_data(['mvrv','sopr','rhodl_ratio','utxo_created_value_sum'],Urls)
ohlc['Volume'] = features['utxo_created_value_sum']

  0%|          | 0/4 [00:00<?, ?it/s]

In [62]:
# import backtesting
# backtesting.set_bokeh_output(notebook=False)
# from numbers import Number
# from inspect import currentframe
# from typing import Sequence, Optional, Union, Callable

# from backtesting import Backtest, Strategy
# from backtesting.lib import crossover
# from backtesting.test import SMA, GOOG

# def EMA(arr: pd.Series, n: int) -> pd.Series:
#     """
#     Returns `n`-period simple moving average of array `arr`.
#     """
#     return pd.Series(arr).ewm(span=n).mean()

    
# class SmaCross(Strategy):
#     def init(self):
#         price = self.data.Close
#         self.ma1 = self.I(EMA, price, 5)
#         self.ma2 = self.I(EMA, price, 60)

#     def next(self):
#         if crossover(self.ma1, self.ma2):
#             self.buy()
#         elif crossover(self.ma2, self.ma1):
#              self.sell()

# bt = Backtest(ohlc.loc['2015':], SmaCross, commission=.002,
#               cash = 100000,exclusive_orders=True,trade_on_close=True)

# stats = bt.run()
# bt.plot()

Row(id='30682', ...)

In [63]:
import numpy as np

def get_X(data):
    """Return model design matrix X"""
    return data.loc[:,data.columns!='Close'].values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [64]:
logR = 100*np.log(ohlc['Close']).rename('logR')

data = pd.concat([ohlc.Close,features],axis=1)
data = data.dropna().copy()
data['Open'] = ohlc['Open']
data['High'] = ohlc['High']
data['Low'] = ohlc['Low']
data=data.rename(columns = {'utxo_created_value_sum':'Volume'})


In [65]:
from backtesting import Backtest, Strategy
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

N_TRAIN = 300


class MLTrainOnceStrategy(Strategy):
    price_delta = .99999 # 0.4%

    def init(self):        
        # Init our model, a SVM classifier
        self.clf = SVC()

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        # Skip the training, in-sample data
        if len(self.data) < N_TRAIN:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)
                
        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
#         for trade in self.trades:
#             if current_time - trade.entry_time > pd.Timedelta('2 days'):
#                 if trade.is_long:
#                     trade.sl = max(trade.sl, low)
#                 else:
#                     trade.sl = min(trade.sl, high)

bt = Backtest(data, 
              MLTrainOnceStrategy, 
              commission=.0002, 
              cash = 10000000,
              exclusive_orders=True,
              trade_on_close=True)
bt.run()
# bt.plot()

Start                     2013-01-01 00:00:00
End                       2022-01-31 00:00:00
Duration                   3317 days 00:00:00
Exposure Time [%]                   90.958409
Equity Final [$]              51037629.660546
Equity Peak [$]               56048932.209058
Return [%]                         410.376297
Buy & Hold Return [%]           290790.239512
Return (Ann.) [%]                   19.638848
Volatility (Ann.) [%]               17.905102
Sharpe Ratio                          1.09683
Sortino Ratio                        2.010126
Calmar Ratio                         0.973577
Max. Drawdown [%]                  -20.171854
Avg. Drawdown [%]                   -2.924264
Max. Drawdown Duration     1055 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   65
Win Rate [%]                        58.461538
Best Trade [%]                      99.959008
Worst Trade [%]                    -51.461942
Avg. Trade [%]                    

In [66]:
bt._results

Start                     2013-01-01 00:00:00
End                       2022-01-31 00:00:00
Duration                   3317 days 00:00:00
Exposure Time [%]                   90.958409
Equity Final [$]              51037629.660546
Equity Peak [$]               56048932.209058
Return [%]                         410.376297
Buy & Hold Return [%]           290790.239512
Return (Ann.) [%]                   19.638848
Volatility (Ann.) [%]               17.905102
Sharpe Ratio                          1.09683
Sortino Ratio                        2.010126
Calmar Ratio                         0.973577
Max. Drawdown [%]                  -20.171854
Avg. Drawdown [%]                   -2.924264
Max. Drawdown Duration     1055 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   65
Win Rate [%]                        58.461538
Best Trade [%]                      99.959008
Worst Trade [%]                    -51.461942
Avg. Trade [%]                    